# **Durée de retour à l'emploi : phase exploratoire**

In [1]:
# nécessité d'importer la librairie xlrd pour permettre à Python de lire Excel (les données INSEE ne sont pas dispobinle en csv)
!pip install xlrd

In [2]:
!pip install openpyxl
# également demandé par Python pour lire Excel

In [3]:
import pandas as pd
import xlrd
import openpyxl
import numpy as np

## 1. Constitution de la BDD OPEN DATA

4 fichiers sont nécessaires à la constitution de cette df :
- 2 indicateurs de dynamisme du marché du travail : taux de chômage et indice de concentration de l'emploi
- 2 codes pour opérer les jointures : code INSEE de la commune (jointure avec les codes postaux) et de la Zone d'emploi (jointure avec les codes communes et le taux de chômage)

**Chargement du fichier de correspondance Code INSEE - Code postal**
- Source : La Poste sur Open Data

> *La base officielle des codes postaux est un jeu de données qui fournit la correspondance entre les codes postaux et les codes INSEE des communes, de France (métropole et DOM), des TOM, ainsi que de MONACO. Le fichier CSV a été mis à jour le 21 mars 2022*

> *La preière version de ce JDD comprenait 39 193 lignes (codes postaux) : suppression depuis Excel des 267 codes postaux 98XXX (Collectivités d’outre-mer non dotées du statut de collectivité territoriale cad non habitées de façon permanente de types : terres australes et antartiques) + MONACO (99) + 5 codes 97XXX non répertoriés > il a donc été réduit à 38926 lignes

- URL : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#resources

In [4]:
communes_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\laposte_hexasmal_5.xlsx", dtype=str)
# avec .dtypes : garder le format string pour toutes les colonnes qui sont des code géographique ou codes postaux (pas de données numériques dans ce fichier)
communes_df

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps
0,01053,BOURG EN BRESSE,01000,NaN,BOURG EN BRESSE,"46.205013838,5.245593605"
1,01053,BOURG EN BRESSE,01000,BROU,BOURG EN BRESSE,"46.205013838,5.245593605"
2,01344,ST DENIS LES BOURG,01000,NaN,ST DENIS LES BOURG,"46.211139007,5.184567751"
3,01165,FRANCHELEINS,01090,AMAREINS,FRANCHELEINS,"46.073642487,4.812560271"
4,01165,FRANCHELEINS,01090,NaN,FRANCHELEINS,"46.073642487,4.812560271"
...,...,...,...,...,...,...
38921,97605,CHICONI,97670,NaN,CHICONI,"-12.821689588,45.113253157"
38922,97614,OUANGANI,97670,COCONI,OUANGANI,"-12.837567293,45.138320778"
38923,97614,OUANGANI,97670,NaN,OUANGANI,"-12.837567293,45.138320778"
38924,97617,TSINGONI,97680,COMBANI,TSINGONI,"-12.782686007,45.13457636"


In [5]:
communes_df.shape

(38926, 6)

In [6]:
# modification et préparation de la table communes_df : en-têtes renommées
communes_df = communes_df.rename(columns = {'code_postal' : 'cp', 'code_commune_insee':'code_commune', 'nom_de_la_commune' : 'commune'})
communes_df = communes_df[['cp', 'code_commune', 'commune']] # ne garder que les colonnes utiles
communes_df

,cp,code_commune,commune
0,01000,01053,BOURG EN BRESSE
1,01000,01053,BOURG EN BRESSE
2,01000,01344,ST DENIS LES BOURG
3,01090,01165,FRANCHELEINS
4,01090,01165,FRANCHELEINS
...,...,...,...
38921,97670,97605,CHICONI
38922,97670,97614,OUANGANI
38923,97670,97614,OUANGANI
38924,97680,97617,TSINGONI


In [7]:
# identifier le nombre de doublons
communes_df.duplicated().sum()

3491

In [8]:
# suppression des doublons
communes_df=communes_df.drop_duplicates(keep="first") 
communes_df # 38 926 lignes - 3 491 doublons = 35 435 entrées

,cp,code_commune,commune
0,01000,01053,BOURG EN BRESSE
2,01000,01344,ST DENIS LES BOURG
3,01090,01165,FRANCHELEINS
6,01090,01169,GENOUILLEUX
7,01090,01183,GUEREINS
...,...,...,...
38919,97660,97603,BANDRELE
38920,97660,97607,DEMBENI
38921,97670,97605,CHICONI
38922,97670,97614,OUANGANI


In [9]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
communes_df.isna().sum()

cp              0
code_commune    0
commune         0
dtype: int64

In [10]:
# vérifier le nombre de valeurs différentes
communes_df.nunique()

cp               6187
code_commune    34997
commune         32684
dtype: int64

In [11]:
#vérifier les types des colonnes : 
communes_df.dtypes

cp              object
code_commune    object
commune         object
dtype: object

**Chargement du fichier base des zones d'emploi Code INSEE - ZE depuis INSEE**
- Source : INSEE - composition communale des zones d'emploi (au format XLSX)
> *Base des zones d'emploi 2020 au 1ᵉʳ janvier 2022. Permet de rattacher chaque commune à sa zone d'emploi, car le taux de chômage actualisé est disponible par zone d'emploi et non par commune*
- URL : https://www.insee.fr/fr/information/4652957

In [12]:
ze_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\ZE2020_au_01-01-2022.xlsx", skiprows = 4, header = 1, sheet_name = 'Composition_communale', dtype=str)
# avec .dtypes : garder le format string pour toutes les colonnes qui sont des code géographique ou codes postaux (pas de données numériques dans ce fichier)
ze_df 

,CODGEO,LIBGEO,ZE2020,LIBZE2020,ZE2020_PARTIE_REG,DEP,REG
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,NaN,01,84
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,NaN,01,84
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,NaN,01,84
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,NaN,01,84
4,01006,Ambléon,8404,Belley,NaN,01,84
...,...,...,...,...,...,...,...
34950,97613,M'Tsangamouji,0601,Mayotte,NaN,976,06
34951,97614,Ouangani,0601,Mayotte,NaN,976,06
34952,97615,Pamandzi,0601,Mayotte,NaN,976,06
34953,97616,Sada,0601,Mayotte,NaN,976,06


In [13]:
# modification et préparation de la table ze_df : en-têtes renommées
# le JDD comprend 34955 lignes (contre 35 435 dans le précédent)
ze_df = ze_df.rename(columns = {"CODGEO":'code_commune2', 
                                          "LIBGEO" : 'commune2',  
                                          "ZE2020" : 'code_ze',
                                         "LIBZE2020" : 'ze'})
ze_df = ze_df[["code_commune2", "commune2", "code_ze", "ze"]] # on ne garde que les colonnes utiles
ze_df

,code_commune2,commune2,code_ze,ze
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône
4,01006,Ambléon,8404,Belley
...,...,...,...,...
34950,97613,M'Tsangamouji,0601,Mayotte
34951,97614,Ouangani,0601,Mayotte
34952,97615,Pamandzi,0601,Mayotte
34953,97616,Sada,0601,Mayotte


In [14]:
# vérification des doublons sur la colonne code_commune2
ze_df.duplicated().sum()

0

In [15]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
ze_df.isna().sum()

code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [16]:
ze_df.dtypes # vérification du type des valeurs dans les colonnes (object = string)

code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

**MERGE sur les 2 premiers JDD CP et codes communes et ZE**
> *Constitution d'un seul jeu avec toutes les informations CP, code commune et nom de la commune, code ZE et nom de la ZE. Dans un second temps, on enlevera les colonnes inutiles (gardées pour vérification).*

> *ATTENTION : le 1er JDD comprend 35 435 lignes. Le second en a uniquement 34 955 (codes communes et codes ZE), donc un différentiel de 480 lignes. Il faudra donc vérifier quels sont codes postaux qui n'ont pas de codes INSEE (par hypothèse les CP d'arrondissements à Paris, Marseille, Lyon)

**Choix du MERGE : on privilégie le JDD le plus complet : sur les communes (35 435 lignes)**
> Pour vérification, on réalise un 1er MERGE ave le JDD sur les ZE (34 955 lignes) en cible, dans lequel on injecte le JDD des communes de 35 435 lignes : on se retrouve avec 3 lignes avec des données manquantes, correspondant à l'équivalent des libéllée et codes de ZE pour Paris, Marseille et Lyon. Cela nous permettra de compléter la donnée que pour ces lignes : c'est le JDD INSEE_df.

> Pour la constitution du JDD INSEE_1 que l'on utilisera, on jointe les communes (JDD le plus complet) en y injectant le JDD ZE : on repère alors 46 données manquantes, que l'on va tenter de compléter.


> Les 2 autres JDD utilisés ensuite jointent sur la ZE (taux de chômage) et le code commune sans les arrondissements.

In [17]:
#MERGE des 2 dataframe à partir de la clé code_commune = code_commune2 avec JDD_ze à gauche (34 955) où il y en a moins que dans le JDD_commune (38 931)
INSEE_df = pd.merge(ze_df, communes_df, how = "left", left_on="code_commune2", right_on="code_commune")
print (INSEE_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne
INSEE_df.isna().sum()

      code_commune2                 commune2 code_ze                      ze  \
0             01001  L'Abergement-Clémenciat    8405         Bourg en Bresse   
1             01002    L'Abergement-de-Varey    8405         Bourg en Bresse   
2             01004        Ambérieu-en-Bugey    8405         Bourg en Bresse   
3             01005      Ambérieux-en-Dombes    8434  Villefranche-sur-Saône   
4             01006                  Ambléon    8404                  Belley   
...             ...                      ...     ...                     ...   
35387         97613            M'Tsangamouji    0601                 Mayotte   
35388         97614                 Ouangani    0601                 Mayotte   
35389         97615                 Pamandzi    0601                 Mayotte   
35390         97616                     Sada    0601                 Mayotte   
35391         97617                 Tsingoni    0601                 Mayotte   

          cp code_commune              

code_commune2    0
commune2         0
code_ze          0
ze               0
cp               3
code_commune     3
commune          3
dtype: int64

Ce JDD ne sera PAS utilisé mais il permet de repérer les valeurs manquantes, et les 3 codes gographiques génériques pour Marseille, Lyon et Paris

In [18]:
# identification et visualisation des 3 lignes avec des valeurs manquantes
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4393,13055,Marseille,9312,Marseille,NaN,NaN,NaN
27442,69123,Lyon,8421,Lyon,NaN,NaN,NaN
29623,75056,Paris,1109,Paris,NaN,NaN,NaN


> Les données manquantes sont les 3 communes avec arrondissements : on garde leur code_commune_2 pour pouvoir compléter ultérieurement les données manquantes du JDD INSEE1_df

In [19]:
# Affiche le nombre de valeurs présentes dans chaque colonne
INSEE_df.count()

code_commune2    35392
commune2         35392
code_ze          35392
ze               35392
cp               35389
code_commune     35389
commune          35389
dtype: int64

In [20]:
# 2ème MERGE des 2 dataframes (inversé) qui sera privilégié à partir de la clé code_commune2 = code_commune avec JDD_commune (38 931) à gauche, car il comporte plus de données que le JDD ze_df (34 955) 
INSEE1_df = pd.merge(communes_df, ze_df, how = "left", left_on="code_commune", right_on="code_commune2")
print (INSEE1_df)

          cp code_commune             commune code_commune2  \
0      01000        01053     BOURG EN BRESSE         01053   
1      01000        01344  ST DENIS LES BOURG         01344   
2      01090        01165        FRANCHELEINS         01165   
3      01090        01169         GENOUILLEUX         01169   
4      01090        01183            GUEREINS         01183   
...      ...          ...                 ...           ...   
35430  97660        97603            BANDRELE         97603   
35431  97660        97607             DEMBENI         97607   
35432  97670        97605             CHICONI         97605   
35433  97670        97614            OUANGANI         97614   
35434  97680        97617            TSINGONI         97617   

                    commune2 code_ze                      ze  
0            Bourg-en-Bresse    8405         Bourg en Bresse  
1      Saint-Denis-lès-Bourg    8405         Bourg en Bresse  
2               Francheleins    8434  Villefranche-sur

In [21]:
# vérifier s'il y a des doublons
INSEE1_df.duplicated().sum()

0

In [22]:
# afficher le nombre de valeurs présentes dans chaque colonne
INSEE1_df.count()

cp               35435
code_commune     35435
commune          35435
code_commune2    35389
commune2         35389
code_ze          35389
ze               35389
dtype: int64

In [23]:
# afficher le nombre de valeurs différentes dans chaque colonne
INSEE1_df.nunique()

cp                6187
code_commune     34997
commune          32684
code_commune2    34952
commune2         32698
code_ze            306
ze                 305
dtype: int64

In [24]:
INSEE1_df.isna().sum() #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne

cp                0
code_commune      0
commune           0
code_commune2    46
commune2         46
code_ze          46
ze               46
dtype: int64

**Mise à jour du JDD INSEE1 : complétion des 46 valeurs manquantes**

Il s'agira d'abord d'identifier les valeurs manquantes du JDD INSEE1 à l'aide du JDD INSEE, puis de remplacer les NaN par les valeurs adéquates

In [25]:
# visualisation des 46 lignes avec des valeurs manquantes
INSEE1_df[INSEE1_df.code_commune2.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4333,13001,13201,MARSEILLE 01,NaN,NaN,NaN,NaN
4334,13002,13202,MARSEILLE 02,NaN,NaN,NaN,NaN
4335,13003,13203,MARSEILLE 03,NaN,NaN,NaN,NaN
4336,13004,13204,MARSEILLE 04,NaN,NaN,NaN,NaN
4337,13005,13205,MARSEILLE 05,NaN,NaN,NaN,NaN
4338,13006,13206,MARSEILLE 06,NaN,NaN,NaN,NaN
4339,13007,13207,MARSEILLE 07,NaN,NaN,NaN,NaN
4340,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4341,13009,13209,MARSEILLE 09,NaN,NaN,NaN,NaN
4342,13010,13210,MARSEILLE 10,NaN,NaN,NaN,NaN


> Les 46 lignes avec données manquantes sont les arrondissements de Marseille, Lyon et Paris

In [26]:
# visualisation des 3 valeurs de destination : codes commune2 + nom2 + code ZE + ze "génériques" pour Marseille, Lyon et Paris
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4393,13055,Marseille,9312,Marseille,NaN,NaN,NaN
27442,69123,Lyon,8421,Lyon,NaN,NaN,NaN
29623,75056,Paris,1109,Paris,NaN,NaN,NaN


In [27]:
# vérification de l'index des lignes Marseille (RAPPEL : borne extérieure exclue du slice)
INSEE1_df[4333:4349]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4333,13001,13201,MARSEILLE 01,NaN,NaN,NaN,NaN
4334,13002,13202,MARSEILLE 02,NaN,NaN,NaN,NaN
4335,13003,13203,MARSEILLE 03,NaN,NaN,NaN,NaN
4336,13004,13204,MARSEILLE 04,NaN,NaN,NaN,NaN
4337,13005,13205,MARSEILLE 05,NaN,NaN,NaN,NaN
4338,13006,13206,MARSEILLE 06,NaN,NaN,NaN,NaN
4339,13007,13207,MARSEILLE 07,NaN,NaN,NaN,NaN
4340,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4341,13009,13209,MARSEILLE 09,NaN,NaN,NaN,NaN
4342,13010,13210,MARSEILLE 10,NaN,NaN,NaN,NaN


In [28]:
# traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Marseille au lieu des valeurs NaN
INSEE1_df.loc[4333:4349,"code_commune2"] = "13055" #affecte le code commune 2
INSEE1_df.loc[4333:4349,"commune2"] = "Marseille" #affecte le commune 2
INSEE1_df.loc[4333:4349,"code_ze"] = "9312" #affecte le code ze
INSEE1_df.loc[4333:4349,"ze"] = "Marseille" #affecte le nom de la ze
INSEE1_df[4333:4349] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4333,13001,13201,MARSEILLE 01,13055,Marseille,9312,Marseille
4334,13002,13202,MARSEILLE 02,13055,Marseille,9312,Marseille
4335,13003,13203,MARSEILLE 03,13055,Marseille,9312,Marseille
4336,13004,13204,MARSEILLE 04,13055,Marseille,9312,Marseille
4337,13005,13205,MARSEILLE 05,13055,Marseille,9312,Marseille
4338,13006,13206,MARSEILLE 06,13055,Marseille,9312,Marseille
4339,13007,13207,MARSEILLE 07,13055,Marseille,9312,Marseille
4340,13008,13208,MARSEILLE 08,13055,Marseille,9312,Marseille
4341,13009,13209,MARSEILLE 09,13055,Marseille,9312,Marseille
4342,13010,13210,MARSEILLE 10,13055,Marseille,9312,Marseille


In [29]:
# vérification de l'index des lignes Lyon (RAPPEL : borne extérieure exclue du slice)
INSEE1_df[27348:27357]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
27348,69001,69381,LYON 01,NaN,NaN,NaN,NaN
27349,69002,69382,LYON 02,NaN,NaN,NaN,NaN
27350,69003,69383,LYON 03,NaN,NaN,NaN,NaN
27351,69004,69384,LYON 04,NaN,NaN,NaN,NaN
27352,69005,69385,LYON 05,NaN,NaN,NaN,NaN
27353,69006,69386,LYON 06,NaN,NaN,NaN,NaN
27354,69007,69387,LYON 07,NaN,NaN,NaN,NaN
27355,69008,69388,LYON 08,NaN,NaN,NaN,NaN
27356,69009,69389,LYON 09,NaN,NaN,NaN,NaN


In [30]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Lyon au lieu des valeurs NaN
INSEE1_df.loc[27348:27357,"code_commune2"] = "69123" #affecte le code commune 2
INSEE1_df.loc[27348:27357,"commune2"] = "Lyon" #affecte le commune 2
INSEE1_df.loc[27348:27357,"code_ze"] = "8421" #affecte le code ze
INSEE1_df.loc[27348:27357,"ze"] = "Lyon" #affecte le nom de la ze
INSEE1_df[27348:27357] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
27348,69001,69381,LYON 01,69123,Lyon,8421,Lyon
27349,69002,69382,LYON 02,69123,Lyon,8421,Lyon
27350,69003,69383,LYON 03,69123,Lyon,8421,Lyon
27351,69004,69384,LYON 04,69123,Lyon,8421,Lyon
27352,69005,69385,LYON 05,69123,Lyon,8421,Lyon
27353,69006,69386,LYON 06,69123,Lyon,8421,Lyon
27354,69007,69387,LYON 07,69123,Lyon,8421,Lyon
27355,69008,69388,LYON 08,69123,Lyon,8421,Lyon
27356,69009,69389,LYON 09,69123,Lyon,8421,Lyon


In [31]:
# vérification de l'index des lignes Paris (RAPPEL : borne extérieure exclue du slice)
INSEE1_df[29648:29669]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
29648,75001,75101,PARIS 01,NaN,NaN,NaN,NaN
29649,75002,75102,PARIS 02,NaN,NaN,NaN,NaN
29650,75003,75103,PARIS 03,NaN,NaN,NaN,NaN
29651,75004,75104,PARIS 04,NaN,NaN,NaN,NaN
29652,75005,75105,PARIS 05,NaN,NaN,NaN,NaN
29653,75006,75106,PARIS 06,NaN,NaN,NaN,NaN
29654,75007,75107,PARIS 07,NaN,NaN,NaN,NaN
29655,75008,75108,PARIS 08,NaN,NaN,NaN,NaN
29656,75009,75109,PARIS 09,NaN,NaN,NaN,NaN
29657,75010,75110,PARIS 10,NaN,NaN,NaN,NaN


In [32]:
# Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Paris au lieu des valeurs NaN
INSEE1_df.loc[29648:29669,"code_commune2"] = "75056" #affecte le code commune 2
INSEE1_df.loc[29648:29669,"commune2"] = "Paris" #affecte le commune 2
INSEE1_df.loc[29648:29669,"code_ze"] = "1109" #affecte le code ze
INSEE1_df.loc[29648:29669,"ze"] = "Paris" #affecte le nom de la ze
INSEE1_df[29648:29669] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
29648,75001,75101,PARIS 01,75056,Paris,1109,Paris
29649,75002,75102,PARIS 02,75056,Paris,1109,Paris
29650,75003,75103,PARIS 03,75056,Paris,1109,Paris
29651,75004,75104,PARIS 04,75056,Paris,1109,Paris
29652,75005,75105,PARIS 05,75056,Paris,1109,Paris
29653,75006,75106,PARIS 06,75056,Paris,1109,Paris
29654,75007,75107,PARIS 07,75056,Paris,1109,Paris
29655,75008,75108,PARIS 08,75056,Paris,1109,Paris
29656,75009,75109,PARIS 09,75056,Paris,1109,Paris
29657,75010,75110,PARIS 10,75056,Paris,1109,Paris


In [33]:
# vérifier que INSEE1_df a toujours le même nombre de lignes (35 435)
INSEE1_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
0,01000,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse
1,01000,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse
2,01090,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône
3,01090,01169,GENOUILLEUX,01169,Genouilleux,8434,Villefranche-sur-Saône
4,01090,01183,GUEREINS,01183,Guéreins,8434,Villefranche-sur-Saône
...,...,...,...,...,...,...,...
35430,97660,97603,BANDRELE,97603,Bandrele,0601,Mayotte
35431,97660,97607,DEMBENI,97607,Dembeni,0601,Mayotte
35432,97670,97605,CHICONI,97605,Chiconi,0601,Mayotte
35433,97670,97614,OUANGANI,97614,Ouangani,0601,Mayotte


In [34]:
#puis afficher le nombre de valeurs manquantes
INSEE1_df.isna().sum()

cp               0
code_commune     0
commune          0
code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [35]:
# afficher le nombre de doublons
INSEE1_df.duplicated().sum()

0

In [36]:
# afficher les types
INSEE1_df.dtypes

cp               object
code_commune     object
commune          object
code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

**Ajout du taux de chômage au JDD INSEE1 (constitution du JDD INSEE2)**

- Source : INSEE.fr

Dans un 1er temps, import du JDD de l'INSEE avec les taux de chômage actualisé au 4ème trimeste 2021

> *Le JDD en format Excel comprend les taux de chômage localisés par zone d'emploi, en moyenne trimestrielle, en France hors Mayotte et Guyane.*

- URL : https://www.insee.fr/fr/statistiques/1893230

Le fichier contient 5 lignes de métadonnées à supprimer et seules les colonnes A (ZE2020 = numéro de la ZE), B (LIBZE2020 = nom de la ZE) et CB (2021-T4 = taux de chômage au 4ème trimestre 2021) sont intéressantes à garder

In [37]:
# import du fichier avec suppression des métadonnées
chomage_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\chomage-zone-t1-2003-t4-2021.xlsx",  skiprows = 4, header = 1, sheet_name = 'txcho_ze')
chomage_df

,ZE2020,LIBZE2020,REG,LIBREG,2003-T1,2003-T2,2003-T3,2003-T4,2004-T1,2004-T2,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,51,Alençon,0,INTERRÉGIONAL,7.2,7.5,7.3,7.5,7.7,7.4,...,8.2,8.1,7.7,7.1,8.7,7.6,7.4,7.5,7.7,7.0
1,52,Arles,0,INTERRÉGIONAL,10.8,11.0,10.7,11.1,11.2,10.8,...,11.1,10.7,10.4,9.8,11.8,10.3,10.3,10.7,10.5,9.9
2,53,Avignon,0,INTERRÉGIONAL,9.4,9.6,9.6,9.7,9.9,9.7,...,10.6,10.3,9.8,9.0,11.3,10.1,10.1,10.1,10.3,9.6
3,54,Beauvais,0,INTERRÉGIONAL,7.1,7.3,7.3,7.8,7.9,7.8,...,8.6,8.4,8.1,7.6,9.3,8.1,8.0,8.1,8.3,7.6
4,55,Bollène-Pierrelatte,0,INTERRÉGIONAL,9.5,9.5,9.4,9.4,9.6,9.5,...,10.7,10.3,9.9,9.0,11.0,9.9,9.9,10.1,10.1,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,9403,Calvi,94,CORSE,11.8,11.7,11.4,11.3,11.2,11.5,...,10.0,10.4,10.3,11.0,13.8,9.1,8.9,9.3,9.3,8.3
298,9404,Corte,94,CORSE,7.6,7.1,7.7,7.5,7.5,7.5,...,7.3,7.1,7.0,6.5,7.8,6.3,5.9,6.2,6.3,5.5
299,9405,Ghisonaccia,94,CORSE,10.3,9.5,9.9,10.3,10.3,10.2,...,9.0,8.8,8.4,7.3,9.6,7.7,7.4,7.7,7.9,7.1
300,9406,Porto-Vecchio,94,CORSE,9.8,9.8,9.9,10.0,10.0,10.0,...,9.1,10.3,10.1,10.8,13.6,9.1,9.1,9.1,8.5,7.7


In [38]:
chomage_df.dtypes # pour vérifier le format des données

ZE2020         int64
LIBZE2020     object
REG            int64
LIBREG        object
2003-T1      float64
              ...   
2020-T4      float64
2021-T1      float64
2021-T2      float64
2021-T3      float64
2021-T4      float64
Length: 80, dtype: object

In [39]:
# vérification des types de données afin de constituer un dictionnaire et de modifier le type à l'import
{key : float for key in chomage_df.columns}

{'ZE2020': float,
 'LIBZE2020': float,
 'REG': float,
 'LIBREG': float,
 '2003-T1': float,
 '2003-T2': float,
 '2003-T3': float,
 '2003-T4': float,
 '2004-T1': float,
 '2004-T2': float,
 '2004-T3': float,
 '2004-T4': float,
 '2005-T1': float,
 '2005-T2': float,
 '2005-T3': float,
 '2005-T4': float,
 '2006-T1': float,
 '2006-T2': float,
 '2006-T3': float,
 '2006-T4': float,
 '2007-T1': float,
 '2007-T2': float,
 '2007-T3': float,
 '2007-T4': float,
 '2008-T1': float,
 '2008-T2': float,
 '2008-T3': float,
 '2008-T4': float,
 '2009-T1': float,
 '2009-T2': float,
 '2009-T3': float,
 '2009-T4': float,
 '2010-T1': float,
 '2010-T2': float,
 '2010-T3': float,
 '2010-T4': float,
 '2011-T1': float,
 '2011-T2': float,
 '2011-T3': float,
 '2011-T4': float,
 '2012-T1': float,
 '2012-T2': float,
 '2012-T3': float,
 '2012-T4': float,
 '2013-T1': float,
 '2013-T2': float,
 '2013-T3': float,
 '2013-T4': float,
 '2014-T1': float,
 '2014-T2': float,
 '2014-T3': float,
 '2014-T4': float,
 '2015-T1': floa

In [40]:
# constitution du dictionnaire dtype pour réimporter les données dans le bon format
chomage_dtype = {
'ZE2020': str,
 'LIBZE2020': str,
 'REG': str,
 'LIBREG': str,
 '2003-T1': float,
 '2003-T2': float,
 '2003-T3': float,
 '2003-T4': float,
 '2004-T1': float,
 '2004-T2': float,
 '2004-T3': float,
 '2004-T4': float,
 '2005-T1': float,
 '2005-T2': float,
 '2005-T3': float,
 '2005-T4': float,
 '2006-T1': float,
 '2006-T2': float,
 '2006-T3': float,
 '2006-T4': float,
 '2007-T1': float,
 '2007-T2': float,
 '2007-T3': float,
 '2007-T4': float,
 '2008-T1': float,
 '2008-T2': float,
 '2008-T3': float,
 '2008-T4': float,
 '2009-T1': float,
 '2009-T2': float,
 '2009-T3': float,
 '2009-T4': float,
 '2010-T1': float,
 '2010-T2': float,
 '2010-T3': float,
 '2010-T4': float,
 '2011-T1': float,
 '2011-T2': float,
 '2011-T3': float,
 '2011-T4': float,
 '2012-T1': float,
 '2012-T2': float,
 '2012-T3': float,
 '2012-T4': float,
 '2013-T1': float,
 '2013-T2': float,
 '2013-T3': float,
 '2013-T4': float,
 '2014-T1': float,
 '2014-T2': float,
 '2014-T3': float,
 '2014-T4': float,
 '2015-T1': float,
 '2015-T2': float,
 '2015-T3': float,
 '2015-T4': float,
 '2016-T1': float,
 '2016-T2': float,
 '2016-T3': float,
 '2016-T4': float,
 '2017-T1': float,
 '2017-T2': float,
 '2017-T3': float,
 '2017-T4': float,
 '2018-T1': float,
 '2018-T2': float,
 '2018-T3': float,
 '2018-T4': float,
 '2019-T1': float,
 '2019-T2': float,
 '2019-T3': float,
 '2019-T4': float,
 '2020-T1': float,
 '2020-T2': float,
 '2020-T3': float,
 '2020-T4': float,
 '2021-T1': float,
 '2021-T2': float,
 '2021-T3': float,
 '2021-T4': float}

In [41]:
# réimport du fichier avec les bons types
chomage_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\chomage-zone-t1-2003-t4-2021.xlsx",  skiprows = 4, header = 1, sheet_name = 'txcho_ze', dtype = chomage_dtype)
chomage_df

,ZE2020,LIBZE2020,REG,LIBREG,2003-T1,2003-T2,2003-T3,2003-T4,2004-T1,2004-T2,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,0051,Alençon,00,INTERRÉGIONAL,7.2,7.5,7.3,7.5,7.7,7.4,...,8.2,8.1,7.7,7.1,8.7,7.6,7.4,7.5,7.7,7.0
1,0052,Arles,00,INTERRÉGIONAL,10.8,11.0,10.7,11.1,11.2,10.8,...,11.1,10.7,10.4,9.8,11.8,10.3,10.3,10.7,10.5,9.9
2,0053,Avignon,00,INTERRÉGIONAL,9.4,9.6,9.6,9.7,9.9,9.7,...,10.6,10.3,9.8,9.0,11.3,10.1,10.1,10.1,10.3,9.6
3,0054,Beauvais,00,INTERRÉGIONAL,7.1,7.3,7.3,7.8,7.9,7.8,...,8.6,8.4,8.1,7.6,9.3,8.1,8.0,8.1,8.3,7.6
4,0055,Bollène-Pierrelatte,00,INTERRÉGIONAL,9.5,9.5,9.4,9.4,9.6,9.5,...,10.7,10.3,9.9,9.0,11.0,9.9,9.9,10.1,10.1,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,9403,Calvi,94,CORSE,11.8,11.7,11.4,11.3,11.2,11.5,...,10.0,10.4,10.3,11.0,13.8,9.1,8.9,9.3,9.3,8.3
298,9404,Corte,94,CORSE,7.6,7.1,7.7,7.5,7.5,7.5,...,7.3,7.1,7.0,6.5,7.8,6.3,5.9,6.2,6.3,5.5
299,9405,Ghisonaccia,94,CORSE,10.3,9.5,9.9,10.3,10.3,10.2,...,9.0,8.8,8.4,7.3,9.6,7.7,7.4,7.7,7.9,7.1
300,9406,Porto-Vecchio,94,CORSE,9.8,9.8,9.9,10.0,10.0,10.0,...,9.1,10.3,10.1,10.8,13.6,9.1,9.1,9.1,8.5,7.7


In [42]:
# vérification des doublons 
chomage_df.duplicated().sum()

0

In [43]:
# vérification des types
chomage_df.dtypes

ZE2020        object
LIBZE2020     object
REG           object
LIBREG        object
2003-T1      float64
              ...   
2020-T4      float64
2021-T1      float64
2021-T2      float64
2021-T3      float64
2021-T4      float64
Length: 80, dtype: object

In [44]:
chomage_df.isna().sum() # vérification de la complétude du fichier CHOMAGE avant le MERGE

ZE2020        0
LIBZE2020     0
REG           0
LIBREG        0
2003-T1      15
             ..
2020-T4       0
2021-T1       0
2021-T2       0
2021-T3       0
2021-T4       0
Length: 80, dtype: int64

> il n'y a aucune valeur manquante dans la clé de jointure (ZE 2020) et dans la données qui nous intéresse (taux de chômage 2021-T4). Les valeurs manquantes sont situées sur la colonne 2003-T1 dont on ne se servira pas

In [45]:
# afficher le nombre de valeurs différentes : permet de vérifier le nombre de valeurs dans la colonne ZE2020 qui servira de clé de jointure
chomage_df.nunique() 

ZE2020       302
LIBZE2020    301
REG           17
LIBREG        17
2003-T1       79
            ... 
2020-T4       87
2021-T1       86
2021-T2       88
2021-T3       89
2021-T4       83
Length: 80, dtype: int64

In [46]:
# afficher le nombre de valeurs différentes : permet de vérifier le nombre de valeurs dans la colonne ze qui servira de clé de jointure
INSEE1_df.nunique() 

cp                6187
code_commune     34997
commune          32684
code_commune2    34954
commune2         32700
code_ze            306
ze                 305
dtype: int64

> ATTENTION : le fichier d'origine comporte 306 codes ZE (clé de jointure) alors que le fichier CHOMAGE n'en comporte que 302. 4 zone d'emploi ne trouveront donc pas de données CHOMAGE.

In [47]:
#MERGE : création de INSEE2_df en concatenant INSEE1_df avec les colonnes ZE2020, LIBZE2020 et 2021-T4 de chomage_df avec clé ZE2020 (numéro de ZE)
INSEE2_df = pd.merge(INSEE1_df, chomage_df, how = "left", left_on="code_ze", right_on="ZE2020")
INSEE2_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,REG,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,01000,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,84,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
1,01000,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse,8405,Bourg en Bresse,84,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
2,01090,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,84,...,6.4,6.3,6.0,5.8,7.0,6.2,6.2,6.1,6.1,5.5
3,01090,01169,GENOUILLEUX,01169,Genouilleux,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,84,...,6.4,6.3,6.0,5.8,7.0,6.2,6.2,6.1,6.1,5.5
4,01090,01183,GUEREINS,01183,Guéreins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,84,...,6.4,6.3,6.0,5.8,7.0,6.2,6.2,6.1,6.1,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35430,97660,97603,BANDRELE,97603,Bandrele,0601,Mayotte,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35431,97660,97607,DEMBENI,97607,Dembeni,0601,Mayotte,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35432,97670,97605,CHICONI,97605,Chiconi,0601,Mayotte,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35433,97670,97614,OUANGANI,97614,Ouangani,0601,Mayotte,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
# ne garder que les colonnes utiles
INSEE2_df = INSEE2_df[["cp", "code_commune", "commune", "code_commune2", "commune2", "code_ze", "ze", "ZE2020", "LIBZE2020", "2021-T4"]]
INSEE2_df.rename (columns = {'2021-T4' : 'chomage_2021T4'}, inplace = True)
INSEE2_df

C:\Users\emman\anaconda3\envs\projet_desu_tre\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4
0,01000,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
1,01000,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
2,01090,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5
3,01090,01169,GENOUILLEUX,01169,Genouilleux,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5
4,01090,01183,GUEREINS,01183,Guéreins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5
...,...,...,...,...,...,...,...,...,...,...
35430,97660,97603,BANDRELE,97603,Bandrele,0601,Mayotte,NaN,NaN,NaN
35431,97660,97607,DEMBENI,97607,Dembeni,0601,Mayotte,NaN,NaN,NaN
35432,97670,97605,CHICONI,97605,Chiconi,0601,Mayotte,NaN,NaN,NaN
35433,97670,97614,OUANGANI,97614,Ouangani,0601,Mayotte,NaN,NaN,NaN


In [49]:
# vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne
INSEE2_df.isna().sum()

cp                 0
code_commune       0
commune            0
code_commune2      0
commune2           0
code_ze            0
ze                 0
ZE2020            43
LIBZE2020         43
chomage_2021T4    43
dtype: int64

In [50]:
# visualiser les 43 lignes avec des valeurs manquantes
INSEE2_df[INSEE2_df.ZE2020.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4
35355,97300,97302,CAYENNE,97302,Cayenne,0301,Est-littoral,NaN,NaN,NaN
35356,97310,97304,KOUROU,97304,Kourou,0303,Savanes,NaN,NaN,NaN
35357,97311,97310,ROURA,97310,Roura,0301,Est-littoral,NaN,NaN,NaN
35358,97312,97358,ST ELIE,97358,Saint-Élie,0303,Savanes,NaN,NaN,NaN
35359,97313,97308,ST GEORGES,97308,Saint-Georges,0301,Est-littoral,NaN,NaN,NaN
35360,97314,97352,SAUL,97352,Saül,0302,Ouest-Guyanais,NaN,NaN,NaN
35361,97315,97312,SINNAMARY,97312,Sinnamary,0303,Savanes,NaN,NaN,NaN
35362,97316,97362,PAPAICHTON,97362,Papaichton,0302,Ouest-Guyanais,NaN,NaN,NaN
35363,97317,97360,APATOU,97360,Apatou,0302,Ouest-Guyanais,NaN,NaN,NaN
35364,97318,97306,MANA,97306,Mana,0302,Ouest-Guyanais,NaN,NaN,NaN


**Données manquantes**

> 43 lignes avec des données manquantes :  il s'agit des communes avec les codes 97XXX, c'est à dire les 4 zones d'emplois d'outremer qui ne figurent pas dans le JDD sur le taux de chômage trimestriel localisé (Savanes, Mayotte, Est-littoral, Ouest-Guyanais).

> Suppression des lignes, en espérant que ces codes ne seront pas demandés dans le fichier "données endogènes".

In [51]:
# suppression des 43 lignes avec des données manquantes (communes d'outremer dont on ne possède pas le taux de chômage)
INSEE2_df = INSEE2_df.dropna()
INSEE_df

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,01400,01001,L ABERGEMENT CLEMENCIAT
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,01640,01002,L ABERGEMENT DE VAREY
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,01500,01004,AMBERIEU EN BUGEY
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,01330,01005,AMBERIEUX EN DOMBES
4,01006,Ambléon,8404,Belley,01300,01006,AMBLEON
...,...,...,...,...,...,...,...
35387,97613,M'Tsangamouji,0601,Mayotte,97650,97613,M TSANGAMOUJI
35388,97614,Ouangani,0601,Mayotte,97670,97614,OUANGANI
35389,97615,Pamandzi,0601,Mayotte,97615,97615,PAMANDZI
35390,97616,Sada,0601,Mayotte,97640,97616,SADA


In [52]:
# vérification de la nouvelle df
INSEE2_df.isna().sum()

cp                0
code_commune      0
commune           0
code_commune2     0
commune2          0
code_ze           0
ze                0
ZE2020            0
LIBZE2020         0
chomage_2021T4    0
dtype: int64

La nouvelle df comprend 35 392 lignes soit 43 de moins (auparavant 34 435) et plus aucune valeur nulle

**Ajout de l'indice de concentration de l'emploi au JDD INSEE2 (constitution du JDD INSEE3)**

- Source : données INSEE publiées par l'Observatoire des Territories (agence nationale de la cohésion des territoire).

Dans un 2nd temps, on ajoute au JDD externe l'indice de concentration d'emploi par territoire communal.

> L'indice de concentration de l'emploi mesure le rapport entre le nombre d’emplois total proposés sur un territoire et le nombre d'actifs occupés (actifs en emploi) qui y résident. Cet indicateur permet d'apprécier la fonction de pôle d'emploi ou la fonction résidentielle d'un espace. Si cet indice est supérieur à 100 alors le nombre d'emplois proposés localement est plus important que le nombre d'actifs qui y résident et qui ont un emploi. Dans ce cas, le territoire considéré occupe une fonction de pôle d'emploi.
> *Le JDD en format Excel comprend l'indicateur de concentration d'emploi (nombre d'emplois pour 100 actifs occupés) pur chaque communes (périmètre 2021) sur une série temporelle allant de 1975 à 2018.*

- URL : https://www.observatoire-des-territoires.gouv.fr/indice-de-concentration-de-lemploi

Le fichier contient 2 feuilles : celle des données "Data" comprend 4 lignes de métadonnées à supprimer et seules les lignes correspondant à l'année 2018 nous intéresent (les lignes 1975, 1982, 1990, 1999, 2008 et 2013 peuvent être supprimées).

In [53]:
# import du fichier avec suppression des métadonnées
emploi_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\insee_rp_hist_1975.xlsx",  skiprows = 3, header = 1, sheet_name = 'Data')
emploi_df

,codgeo,libgeo,an,ind_concentr_emp
0,01001,L'Abergement-Clémenciat,1975,48.8
1,01001,L'Abergement-Clémenciat,1982,33.3
2,01001,L'Abergement-Clémenciat,1990,25.0
3,01001,L'Abergement-Clémenciat,1999,30.4
4,01001,L'Abergement-Clémenciat,2008,18.1
...,...,...,...,...
244750,97617,Tsingoni,1990,NaN
244751,97617,Tsingoni,1999,NaN
244752,97617,Tsingoni,2008,NaN
244753,97617,Tsingoni,2013,NaN


In [54]:
# droper les lignes correspondant aux années 1975, 1982, 1990, 1999, 2008 et 2013 et garder que l'année 2018
year_df_idx=emploi_df[emploi_df["an"]<2018].index
emploi_df = emploi_df.drop(year_df_idx)
emploi_df

,codgeo,libgeo,an,ind_concentr_emp
6,01001,L'Abergement-Clémenciat,2018,27.0
13,01002,L'Abergement-de-Varey,2018,14.6
20,01004,Ambérieu-en-Bugey,2018,132.2
27,01005,Ambérieux-en-Dombes,2018,30.6
34,01006,Ambléon,2018,12.0
...,...,...,...,...
244726,97613,M'Tsangamouji,2018,NaN
244733,97614,Ouangani,2018,NaN
244740,97615,Pamandzi,2018,NaN
244747,97616,Sada,2018,NaN


> Il ne reste que 34 965 colonnes équivalent aux années 2018 (suppression de 209 790 lignes soit 85% correspondant aux années 1975, 1982, 1990, 1999, 2008 et 2013)

**MERGE avec le JDD INSEE2 et le JDD comportant les indices de concentration d'emploi et création de INSEE3_df**

On va désormais injecter ces 34 965 indices de concentration d'emploi dans la BDD INSEE2 comportant 35 492 communes : soit un différentiel de 427 lignes.

Manque-t-il 427 communes dans le JDD emploi_df ?

In [55]:
#modification du format de la colonne code géo pour pouvoir effectuer le MERGE
emploi_df['codgeo'] = emploi_df['codgeo'].astype(str)

In [56]:
#MERGE : création de INSEE3_df en conctenant INSEE2_df avec emploi_df avec clé code_commune2 (comportant les villes sans les arrondissements) et codgeo (numéro de commune)
INSEE3_df = pd.merge(INSEE2_df, emploi_df, how = "left", left_on="code_commune2", right_on="codgeo")
INSEE3_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4,codgeo,libgeo,an,ind_concentr_emp
0,01000,01053,BOURG EN BRESSE,01053,Bourg-en-Bresse,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5,01053,Bourg-en-Bresse,2018,218.4
1,01000,01344,ST DENIS LES BOURG,01344,Saint-Denis-lès-Bourg,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5,01344,Saint-Denis-lès-Bourg,2018,53.1
2,01090,01165,FRANCHELEINS,01165,Francheleins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5,01165,Francheleins,2018,18.6
3,01090,01169,GENOUILLEUX,01169,Genouilleux,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5,01169,Genouilleux,2018,14.8
4,01090,01183,GUEREINS,01183,Guéreins,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5,01183,Guéreins,2018,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35387,97450,97414,ST LOUIS,97414,Saint-Louis,0404,Le Sud,0404,Le Sud,21.2,97414,Saint-Louis,2018,88.2
35388,97460,97415,ST PAUL,97415,Saint-Paul,0402,L'Ouest,0402,L'Ouest,18.0,97415,Saint-Paul,2018,87.6
35389,97470,97410,ST BENOIT,97410,Saint-Benoît,0401,L'Est,0401,L'Est,21.2,97410,Saint-Benoît,2018,103.9
35390,97480,97412,ST JOSEPH,97412,Saint-Joseph,0404,Le Sud,0404,Le Sud,21.2,97412,Saint-Joseph,2018,81.2


Il y a le même nombre de lignes : 35 392.

In [57]:
# vérifier les données manquantes de la nouvelle df
INSEE3_df.isna().sum()

cp                  0
code_commune        0
commune             0
code_commune2       0
commune2            0
code_ze             0
ze                  0
ZE2020              0
LIBZE2020           0
chomage_2021T4      0
codgeo              0
libgeo              0
an                  0
ind_concentr_emp    8
dtype: int64

> Les indices de concentration d'emploi sont manquants pour 8 communes

In [58]:
# identification et visualisation des 8 lignes avec des valeurs manquantes
INSEE3_df[INSEE3_df.ind_concentr_emp.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4,codgeo,libgeo,an,ind_concentr_emp
1755,05140,05066,LA HAUTE BEAUME,05066,La Haute-Beaume,9310,Gap,9310,Gap,6.7,05066,La Haute-Beaume,2018,NaN
19040,51800,51470,ROUVROY RIPONT,51470,Rouvroy-Ripont,4402,Châlons-en-Champagne,4402,Châlons-en-Champagne,6.2,51470,Rouvroy-Ripont,2018,NaN
20355,55100,55039,BEAUMONT EN VERDUNOIS,55039,Beaumont-en-Verdunois,4426,Verdun,4426,Verdun,8.3,55039,Beaumont-en-Verdunois,2018,NaN
20359,55100,55050,BEZONVAUX,55050,Bezonvaux,4426,Verdun,4426,Verdun,8.3,55050,Bezonvaux,2018,NaN
20365,55100,55139,CUMIERES LE MORT HOMME,55139,Cumières-le-Mort-Homme,4426,Verdun,4426,Verdun,8.3,55139,Cumières-le-Mort-Homme,2018,NaN
20368,55100,55189,FLEURY DEVANT DOUAUMONT,55189,Fleury-devant-Douaumont,4426,Verdun,4426,Verdun,8.3,55189,Fleury-devant-Douaumont,2018,NaN
20371,55100,55239,HAUMONT PRES SAMOGNEUX,55239,Haumont-près-Samogneux,4426,Verdun,4426,Verdun,8.3,55239,Haumont-près-Samogneux,2018,NaN
20373,55100,55307,LOUVEMONT COTE DU POIVRE,55307,Louvemont-Côte-du-Poivre,4426,Verdun,4426,Verdun,8.3,55307,Louvemont-Côte-du-Poivre,2018,NaN


8 communes n'ont pas d'indicateur INDICE DE CONCENTRATION DE L'EMPLOI : si ces codes figurent dans la BDD des salariés, nous les supprimerons.

## 2. Consolidation de la BDD "Data exogènes"

Il est maintenant possible de ne garder que les colonnes utiles pour réaliser le MERGE avec le JDD des salariés, à savoir :
- l'index
- le code postal (pour réaliser la jointure)
- le code commune et le code commune 2 (pour voir lequel offrira la meilleure jointure - cad tenant compte ou non des arrondissements)
- le taux de chômage
- l'indicateur de concentration d'emploi
- le nom de la commune et celui de la zone d'emploi en cas d'utilité pour la partie descriptive

In [59]:
# modification et préparation de la table INSEE3_df : en-têtes renommées
INSEE3_df = INSEE3_df.rename(columns = {"code_commune":'codgeo1', 
                                          "code_commune2" : 'codgeo2',  
                                          "codgeo" : 'codgeo3',
                                         "ind_concentr_emp" : 'concentr_emp2018'})
INSEE4_df = INSEE3_df[["cp", "codgeo1", "codgeo2", "codgeo3", "commune", "ze", "chomage_2021T4", "concentr_emp2018"]] # on ne garde que les colonnes utiles 
INSEE4_df

,cp,codgeo1,codgeo2,codgeo3,commune,ze,chomage_2021T4,concentr_emp2018
0,01000,01053,01053,01053,BOURG EN BRESSE,Bourg en Bresse,5.5,218.4
1,01000,01344,01344,01344,ST DENIS LES BOURG,Bourg en Bresse,5.5,53.1
2,01090,01165,01165,01165,FRANCHELEINS,Villefranche-sur-Saône,5.5,18.6
3,01090,01169,01169,01169,GENOUILLEUX,Villefranche-sur-Saône,5.5,14.8
4,01090,01183,01183,01183,GUEREINS,Villefranche-sur-Saône,5.5,73.0
...,...,...,...,...,...,...,...,...
35387,97450,97414,97414,97414,ST LOUIS,Le Sud,21.2,88.2
35388,97460,97415,97415,97415,ST PAUL,L'Ouest,18.0,87.6
35389,97470,97410,97410,97410,ST BENOIT,L'Est,21.2,103.9
35390,97480,97412,97412,97412,ST JOSEPH,Le Sud,21.2,81.2


## 3. Import de la BDD "Data endogène"

La BDD des salariés a été constituée de manière fictive mais elle correspond à la typologie réelle d'une BDD comportant des datas sur le RETOUR A L'EMPLOI :
- le nom et prénom (ou matricule)
- l'âge, le genre (parfois l'ancienneté dans l'entreprise mais pas dans ce cas)
- la CSP (parfois l'intitulé métier et/ou le iveau de qualification, mais pas dans ce cas)
- le lieu de résidence (code postal, parfois le département)
- la taille et le secteur d'activité de l'entreprise d'origine
- le temps de retour à l'emploi (ou de solution trouvée)

Une fois cette BDD importée, on pourra y injecter les 2 indicateurs exogènes TAUX DE CHOMAGE et INDICE DE CONCENTRATION D'EMPLOI pour ajouter des données qualitatives à la BDD.

In [60]:
# import de la BDD endogène RAE (Temps de Retour à l'Emploi)
tre_df = pd.read_csv (r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\raw\fake_RAE_v_3.csv")
tre_df

,id,genre,prenom,nom,date_de_naissance,age,csp,nbr_de_salaries,code_naf,ville,code_commune,code_postal,dept,pop,duree_rae
0,1,h,Daniel,VOL,1982-12-24,39,Employé,208,80,Reinhardsmunster,67391,67440,67,455,351
1,2,f,Valentine,GAUGET PETIT,1990-01-10,32,Cadre supérieur,324,80,Villefranche-de-Conflent,66223,66500,66,218,215
2,3,f,Angèle,BELAOUAZZA,2001-05-21,21,Cadre supérieur,196,93,Saint-Gervais-les-Bains,74236,74170,74,5702,270
3,4,f,Jeanine,HERMOSA GUIRADO,1983-04-27,39,Cadre,538,52,La Verpillière,38537,38290,38,7313,166
4,5,h,Amaury,HALBERT,1987-03-04,35,Ouvrier,930,17,Montpellier,34172,34090,34,288600,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,49995,h,Ghislain,ASATEKIN,1992-10-08,29,Technicien Maîtrise,2965,37,Toulouse,31555,31000,31,484809,0
49995,49996,h,Julien,PAUL DUBAIL,1989-01-31,33,Cadre,452,76,Marseille 13e Arrondissement,13213,13013,13,92728,168
49996,49997,h,Hercule,UKOBIZABA,1992-03-22,30,Cadre,5010,80,Néris-les-Bains,03195,3310,03,2646,55
49997,49998,f,Vienne,FLAUJAT,1967-06-24,54,Cadre supérieur,586,20,Plateau d'Hauteville,01185,1110,01,5031,33


In [61]:
# vérfication du type des colonnes de la BDD RAE
tre_df.dtypes

id                    int64
genre                object
prenom               object
nom                  object
date_de_naissance    object
age                   int64
csp                  object
nbr_de_salaries       int64
code_naf              int64
ville                object
code_commune         object
code_postal           int64
dept                 object
pop                   int64
duree_rae             int64
dtype: object

In [62]:
# constitution du dictionnaire des types de la BDD 
tre_dtypes = {
'id': int,
'genre' : str,
'prenom' : str,
'nom' : str,
'age' : int,
'csp' : str,
'nbr_de_salaries' : int,
'code_naf' : int,
'ville' : str,
'code_commune' : str,
'code_postal' : str,
'dept' : str,
'pop' : int,
'duree_rae' : int}

In [63]:
# réimport de la BDD avec les bons types
tre_df = pd.read_csv (r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\raw\fake_RAE_v_3.csv", dtype = tre_dtypes)
tre_df

,id,genre,prenom,nom,date_de_naissance,age,csp,nbr_de_salaries,code_naf,ville,code_commune,code_postal,dept,pop,duree_rae
0,1,h,Daniel,VOL,1982-12-24,39,Employé,208,80,Reinhardsmunster,67391,67440,67,455,351
1,2,f,Valentine,GAUGET PETIT,1990-01-10,32,Cadre supérieur,324,80,Villefranche-de-Conflent,66223,66500,66,218,215
2,3,f,Angèle,BELAOUAZZA,2001-05-21,21,Cadre supérieur,196,93,Saint-Gervais-les-Bains,74236,74170,74,5702,270
3,4,f,Jeanine,HERMOSA GUIRADO,1983-04-27,39,Cadre,538,52,La Verpillière,38537,38290,38,7313,166
4,5,h,Amaury,HALBERT,1987-03-04,35,Ouvrier,930,17,Montpellier,34172,34090,34,288600,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,49995,h,Ghislain,ASATEKIN,1992-10-08,29,Technicien Maîtrise,2965,37,Toulouse,31555,31000,31,484809,0
49995,49996,h,Julien,PAUL DUBAIL,1989-01-31,33,Cadre,452,76,Marseille 13e Arrondissement,13213,13013,13,92728,168
49996,49997,h,Hercule,UKOBIZABA,1992-03-22,30,Cadre,5010,80,Néris-les-Bains,03195,03310,03,2646,55
49997,49998,f,Vienne,FLAUJAT,1967-06-24,54,Cadre supérieur,586,20,Plateau d'Hauteville,01185,01110,01,5031,33


In [64]:
# re-vérification du type
tre_df.dtypes

id                    int32
genre                object
prenom               object
nom                  object
date_de_naissance    object
age                   int32
csp                  object
nbr_de_salaries       int32
code_naf              int32
ville                object
code_commune         object
code_postal          object
dept                 object
pop                   int32
duree_rae             int32
dtype: object

In [65]:
#MERGE depuis le CODE GEOGRAPHIQUE
df = pd.merge(tre_df, INSEE4_df, how = "left", left_on="code_commune", right_on="codgeo1")
df

,id,genre,prenom,nom,date_de_naissance,age,csp,nbr_de_salaries,code_naf,ville,...,pop,duree_rae,cp,codgeo1,codgeo2,codgeo3,commune,ze,chomage_2021T4,concentr_emp2018
0,1,h,Daniel,VOL,1982-12-24,39,Employé,208,80,Reinhardsmunster,...,455,351,67440,67391,67391,67391,REINHARDSMUNSTER,Sarrebourg,5.4,12.7
1,2,f,Valentine,GAUGET PETIT,1990-01-10,32,Cadre supérieur,324,80,Villefranche-de-Conflent,...,218,215,66500,66223,66223,66223,VILLEFRANCHE DE CONFLENT,Perpignan,12.1,179.5
2,3,f,Angèle,BELAOUAZZA,2001-05-21,21,Cadre supérieur,196,93,Saint-Gervais-les-Bains,...,5702,270,74170,74236,74236,74236,ST GERVAIS LES BAINS,Le Mont Blanc,4.1,68.9
3,4,f,Jeanine,HERMOSA GUIRADO,1983-04-27,39,Cadre,538,52,La Verpillière,...,7313,166,38290,38537,38537,38537,LA VERPILLIERE,Bourgoin-Jallieu,6.3,91.1
4,5,h,Amaury,HALBERT,1987-03-04,35,Ouvrier,930,17,Montpellier,...,288600,45,34000,34172,34172,34172,MONTPELLIER,Montpellier,9.8,151.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89914,49995,h,Ghislain,ASATEKIN,1992-10-08,29,Technicien Maîtrise,2965,37,Toulouse,...,484809,0,31500,31555,31555,31555,TOULOUSE,Toulouse,7.4,152.4
89915,49996,h,Julien,PAUL DUBAIL,1989-01-31,33,Cadre,452,76,Marseille 13e Arrondissement,...,92728,168,13013,13213,13055,13055,MARSEILLE 13,Marseille,9.6,112.0
89916,49997,h,Hercule,UKOBIZABA,1992-03-22,30,Cadre,5010,80,Néris-les-Bains,...,2646,55,03310,03195,03195,03195,NERIS LES BAINS,Montluçon,8.8,101.0
89917,49998,f,Vienne,FLAUJAT,1967-06-24,54,Cadre supérieur,586,20,Plateau d'Hauteville,...,5031,33,01110,01185,01185,01185,PLATEAU D HAUTEVILLE,Belley,5.5,118.2


**MEMENTO** : le MERGE avec le CODE POSTAL fonctionne, mais il y a 69 lignes avec des données manquantes.

Nous avons donc privilégié le MERGE avec le codgeo1, car il ne génère AUCUNE donnée manquante.

In [66]:
# vérifier les données manquantes de la nouvelle df
df.isna().sum()

id                   0
genre                0
prenom               0
nom                  0
date_de_naissance    0
age                  0
csp                  0
nbr_de_salaries      0
code_naf             0
ville                0
code_commune         0
code_postal          0
dept                 0
pop                  0
duree_rae            0
cp                   0
codgeo1              0
codgeo2              0
codgeo3              0
commune              0
ze                   0
chomage_2021T4       0
concentr_emp2018     0
dtype: int64

**Traitement des doublons**

On est passé de 49 999 lignes à 89 919 : soit 39 920 lignes en plus, potentiellement des doublons.

In [67]:
# Somme de la série pour pouvoir compter combien de valeurs sont dupliquées. 
# Le paramètre "first" au lieu de False permet d'identifier tous les doublons sauf la 1ère ligne que l'on veut garder
df.duplicated(subset=['id', 'prenom', 'nom'], keep="first").sum()

39920

> On voit qu'il y a 39 920 doublons identifiés si l'on garde la première occurence, ce qui correspond exactement à la taille de la BDD endogène d'origine. 

On va donc pouvoir procéder à l'élimination de ces doublons.

IMPOSSIBLE de dire pourquoi cela a généré cesdoublons mais on va procédé à leur élimination.

In [68]:
# suppresion des doublons
df_unique=df.drop_duplicates(subset =['id', 'prenom', 'nom'],keep="first") 
df_unique

,id,genre,prenom,nom,date_de_naissance,age,csp,nbr_de_salaries,code_naf,ville,...,pop,duree_rae,cp,codgeo1,codgeo2,codgeo3,commune,ze,chomage_2021T4,concentr_emp2018
0,1,h,Daniel,VOL,1982-12-24,39,Employé,208,80,Reinhardsmunster,...,455,351,67440,67391,67391,67391,REINHARDSMUNSTER,Sarrebourg,5.4,12.7
1,2,f,Valentine,GAUGET PETIT,1990-01-10,32,Cadre supérieur,324,80,Villefranche-de-Conflent,...,218,215,66500,66223,66223,66223,VILLEFRANCHE DE CONFLENT,Perpignan,12.1,179.5
2,3,f,Angèle,BELAOUAZZA,2001-05-21,21,Cadre supérieur,196,93,Saint-Gervais-les-Bains,...,5702,270,74170,74236,74236,74236,ST GERVAIS LES BAINS,Le Mont Blanc,4.1,68.9
3,4,f,Jeanine,HERMOSA GUIRADO,1983-04-27,39,Cadre,538,52,La Verpillière,...,7313,166,38290,38537,38537,38537,LA VERPILLIERE,Bourgoin-Jallieu,6.3,91.1
4,5,h,Amaury,HALBERT,1987-03-04,35,Ouvrier,930,17,Montpellier,...,288600,45,34000,34172,34172,34172,MONTPELLIER,Montpellier,9.8,151.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89909,49995,h,Ghislain,ASATEKIN,1992-10-08,29,Technicien Maîtrise,2965,37,Toulouse,...,484809,0,31000,31555,31555,31555,TOULOUSE,Toulouse,7.4,152.4
89915,49996,h,Julien,PAUL DUBAIL,1989-01-31,33,Cadre,452,76,Marseille 13e Arrondissement,...,92728,168,13013,13213,13055,13055,MARSEILLE 13,Marseille,9.6,112.0
89916,49997,h,Hercule,UKOBIZABA,1992-03-22,30,Cadre,5010,80,Néris-les-Bains,...,2646,55,03310,03195,03195,03195,NERIS LES BAINS,Montluçon,8.8,101.0
89917,49998,f,Vienne,FLAUJAT,1967-06-24,54,Cadre supérieur,586,20,Plateau d'Hauteville,...,5031,33,01110,01185,01185,01185,PLATEAU D HAUTEVILLE,Belley,5.5,118.2


In [69]:
# vérifier les données manquantes de la nouvelle df
df_unique.isna().sum()

id                   0
genre                0
prenom               0
nom                  0
date_de_naissance    0
age                  0
csp                  0
nbr_de_salaries      0
code_naf             0
ville                0
code_commune         0
code_postal          0
dept                 0
pop                  0
duree_rae            0
cp                   0
codgeo1              0
codgeo2              0
codgeo3              0
commune              0
ze                   0
chomage_2021T4       0
concentr_emp2018     0
dtype: int64

In [70]:
# données techniques sur la df
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49999 entries, 0 to 89918
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 49999 non-null  int32  
 1   genre              49999 non-null  object 
 2   prenom             49999 non-null  object 
 3   nom                49999 non-null  object 
 4   date_de_naissance  49999 non-null  object 
 5   age                49999 non-null  int32  
 6   csp                49999 non-null  object 
 7   nbr_de_salaries    49999 non-null  int32  
 8   code_naf           49999 non-null  int32  
 9   ville              49999 non-null  object 
 10  code_commune       49999 non-null  object 
 11  code_postal        49999 non-null  object 
 12  dept               49999 non-null  object 
 13  pop                49999 non-null  int32  
 14  duree_rae          49999 non-null  int32  
 15  cp                 49999 non-null  object 
 16  codgeo1            499

> Nous disposons désormais d'une BDD de 49 999 individus avec un id spécifique, 2252 lignes ayant des données manquantes, et 23 colonnes.

## 4. Finalisation et export de la BDD

Pour finaliser le JDD, nous allons pouvoir supprimer les colonnes inutiles : 
- date_de_naissance (l'age suffira), 
- le code NAF (la répartition en 88 NAF est trop compliquée à gérer en ONE HOT ENCODING donc on l'élimine pour cette fois), 
- le département, 
- la population totale, 
- le cp et les 3 codgeo, 
- la commune (on ne garde que la ville).

Nous pourrons ensuite l'exporter pour l'utiliser dans le prochain notebook consacré à la statistique descriptive.

In [71]:
# suppression des colonnes inutiles
df2 = df_unique[["id", "genre", "prenom", "nom", "age", "csp", "nbr_de_salaries", "ville", "duree_rae", "ze", "chomage_2021T4", "concentr_emp2018"]] # on ne garde que les colonnes utiles 
df2

,id,genre,prenom,nom,age,csp,nbr_de_salaries,ville,duree_rae,ze,chomage_2021T4,concentr_emp2018
0,1,h,Daniel,VOL,39,Employé,208,Reinhardsmunster,351,Sarrebourg,5.4,12.7
1,2,f,Valentine,GAUGET PETIT,32,Cadre supérieur,324,Villefranche-de-Conflent,215,Perpignan,12.1,179.5
2,3,f,Angèle,BELAOUAZZA,21,Cadre supérieur,196,Saint-Gervais-les-Bains,270,Le Mont Blanc,4.1,68.9
3,4,f,Jeanine,HERMOSA GUIRADO,39,Cadre,538,La Verpillière,166,Bourgoin-Jallieu,6.3,91.1
4,5,h,Amaury,HALBERT,35,Ouvrier,930,Montpellier,45,Montpellier,9.8,151.3
...,...,...,...,...,...,...,...,...,...,...,...,...
89909,49995,h,Ghislain,ASATEKIN,29,Technicien Maîtrise,2965,Toulouse,0,Toulouse,7.4,152.4
89915,49996,h,Julien,PAUL DUBAIL,33,Cadre,452,Marseille 13e Arrondissement,168,Marseille,9.6,112.0
89916,49997,h,Hercule,UKOBIZABA,30,Cadre,5010,Néris-les-Bains,55,Montluçon,8.8,101.0
89917,49998,f,Vienne,FLAUJAT,54,Cadre supérieur,586,Plateau d'Hauteville,33,Belley,5.5,118.2


On a réduit la dataframe de 23 à 12 colonnes et et aucune ligne ne comporte de données manquantes.

La dataframe est désormais propre (pas de données manquantes) et elle comprend uniquement les indicateurs souhaités pour l'analyse, au bon format.

Nous avons retiré les colonnes d'identification de type CP ou code géographiques qui pourraient être considérés comme des données numériques, ainsi que la colonne catégorielle NAF-APE trop complexe à traiter (OHE).

In [72]:
# export de la dataframe afin de la réimporter dans la notebook consacré à la statistique descriptive.
df2.to_csv(r'C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\processed\tre_dataframe.csv', index = False)